In [2]:
import mdtraj as md
import numpy as np
import itertools
import parmed as parm
import nglview as nv
import pyemma.coordinates as coor

from scipy import misc, signal
import matplotlib.pyplot as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/home/vladas/software/anaconda/lib/python2.7/site-packages/pyemma/__init__.py:91: UserWarning: You are not using the latest release of PyEMMA. Latest is 2.4, you have 2.3.2.
  .format(latest=latest, current=current), category=UserWarning)


# Density grids

In [3]:
def writeDX(filename, H, edges):
    """
    Write DX potential maps based on formating by VMD:
    http://www.ks.uiuc.edu/Research/vmd/plugins/molfile/dxplugin.html
    
    Parameters
    -------
    H : ndarray
        The multidimensional histogram of sample x. See normed and weights
        for the different possible semantics.
    edges : list
        A list of D arrays describing the bin edges for each dimension.
    
    """
    comments = "Grid generated by writeDX python function by v.oleinikovas@gmail.com"
    name = "test"
    
    xn, yn, zn = np.shape(H)
    xcenters = 0.5 * (edges[0][1:] + edges[0][:-1])
    ycenters = 0.5 * (edges[1][1:] + edges[1][:-1])
    zcenters = 0.5 * (edges[2][1:] + edges[2][:-1])
    xorg, yorg, zorg = xcenters[0], ycenters[0], zcenters[0]
    xdel, ydel, zdel = xcenters[1] - xcenters[0], ycenters[1] - ycenters[0], zcenters[1] - zcenters[0]
    
    header =\
    "# Comments\n# %s\n" % comments +\
    "object 1 class gridpositions counts %i %i %i\n" % (xn, yn, zn) +\
    "origin %.2f %.2f %.2f\n" % (xorg, yorg, zorg) +\
    "delta %.2f 0 0\n" % xdel +\
    "delta 0 %.2f 0\n" % ydel +\
    "delta 0 0 %.2f\n" % zdel +\
    "object 2 class gridconnections counts %i %i %i\n" % (xn, yn, zn) +\
    "object 3 class array type double rank 0 items %i data follows\n" % (xn*yn*zn)
    
    with open(filename, "w") as outfile:
        outfile.write(header)
        # Grid data follows, with three values per line, ordered z fast, y medium, and x slow.
        
        # flatten histogram data
        # C' means to read / write the elements using C-like index order, with the last axis
        # index changing fastest, back to the first axis index changing slowest.
        H = np.reshape(H, xn*yn*zn, order="C")
        
        for i in np.arange(xn*yn*zn):
            if i % 3 == 2: # newline after the fird
                outfile.write("%.5e\n" % H[i])
            else:
                outfile.write("%.5e " % H[i])
        outfile.write("\nobject \"%s\" class field\n" % name)

In [1]:
# http://www.ks.uiuc.edu/Research/vmd/plugins/volmapgui/

In [511]:
def gaussian(x, sig):
    return np.exp(-x**2 / (2 * sig**2)) #/ np.sqrt(2*np.pi*sig**2)

def calc_volmap(trajin, topin, selection, edges, chunk_size=100, norm=True):
    """
    Take trajectory and provide a density map.
    Relies on pyemma package to read mdtraj in chunks to avoid memory problems.
    """
    start = time.time()
    
    # get the grid
    xgrid, ygrid, zgrid = edges
    # build the histogram master template
    H = np.empty((int((xgrid[-1]-xgrid[0])/(xgrid[1]-xgrid[0])),
                  int((ygrid[-1]-ygrid[0])/(ygrid[1]-ygrid[0])),
                  int((zgrid[-1]-zgrid[0])/(zgrid[1]-zgrid[0]))), dtype=float)
    
    feat = coor.featurizer(topin) # define topology
    # just use selection xyz-coordinates
    feat.add_selection(feat.topology.select(selection))
    # loading from disk !
    inp = coor.source(trajin, feat, chunk_size=chunk_size)
    print 'trajectory length = ',inp.trajectory_length(0)
    print 'number of dimension = ',inp.dimension()
    N_atoms = inp.dimension()/3 # reads x1,y1,z1, x2,y2,z2, ...

    # iterate over trajectory chucks
    for _, chunk in inp.data_producer:
        data = np.reshape(chunk, (len(chunk)*N_atoms, 3))
        h,_ = np.histogramdd(data*10, bins=edges) # *10 for nm to A !
        # sum to the master template
        H += h
        break
    
    if norm:
        mask_nonzero = H!=0
        # smoothen the volmap by blurring
        radius = 1.4 # use "water radius"
        print("N.B. Gaussian bluring assumes 1 A grid resolution!")
        x = np.linspace(-3, 3, 7) # stepx = res = 1 A
        gaus = gaussian(x, radius)
        gaus3d = np.multiply.outer(gaus, np.multiply.outer(gaus, gaus))
        # apply gaussian "blurring" to provide smoother surface
        H = signal.fftconvolve(H, gaus3d, mode='same')
        # normalize to std.
        H /= H[mask_nonzero].std() # remove zeroes from std calc.
    
    print("This took: %.3f s." % (time.time() - start))
    
    return H

In [523]:
topin = "/home/vladas/Dropbox/IPython_notebooks/Fragmaps_JACS2016/TEM1/ref_1JWP+WAT+BEN.pdb"
trajin = "/home/vladas/mnt/IB-server/scratch/TEM1/HREX+FRAGS/TEM1+BEN/traj_full_fixed/traj0_aligned_full.xtc"
selection = "resname BEN and name DM"

res = 0.1
xmin,xmax = -45.0, 65.0
ymin,ymax = -60.0, 60.0
zmin,zmax = -25.0, 95.0

edges = [np.arange(xmin,xmax+res,res), np.arange(ymin,ymax+res,res), np.arange(zmin,zmax+res,res)]

xgrid, ygrid, zgrid = edges
# build the histogram master template
# H = np.empty((int((xgrid[-1]-xgrid[0])/(xgrid[1]-xgrid[0])),
#               int((ygrid[-1]-ygrid[0])/(ygrid[1]-ygrid[0])),
#               int((zgrid[-1]-zgrid[0])/(zgrid[1]-zgrid[0]))), dtype=float)

# H = calc_volmap(trajin, topin, selection, edges, chunk_size=100)

# writeDX("test3.dx", H, edges)

In [ ]:
# Is the normalization by std consistent enough to allow direct comparison

In [524]:
print(int((xgrid[-1]-xgrid[0])/(xgrid[1]-xgrid[0])),\
      int((ygrid[-1]-ygrid[0])/(ygrid[1]-ygrid[0])),\
      int((zgrid[-1]-zgrid[0])/(zgrid[1]-zgrid[0])))

(1100, 1200, 1200)


In [499]:
topin = "/home/vladas/Dropbox/IPython_notebooks/Fragmaps_JACS2016/TEM1/ref_1JWP+WAT+BEN.pdb"
selection = "resname BEN and name DM"

xgrid, ygrid, zgrid = edges
# build the histogram master template
Href = np.empty((int((xgrid[-1]-xgrid[0])/(xgrid[1]-xgrid[0])),
              int((ygrid[-1]-ygrid[0])/(ygrid[1]-ygrid[0])),
              int((zgrid[-1]-zgrid[0])/(zgrid[1]-zgrid[0]))), dtype=float)

for i in np.arange(32):
    trajin = "/home/vladas/mnt/IB-server/scratch/TEM1/multi_NVT/protein+BEN/fixed_traj/traj%s_aligned_full.xtc" % i
    href = calc_volmap(trajin, topin, selection, edges, chunk_size=100, norm=False)
    Href += href

mask_nonzero = Href!=0
# smoothen the volmap by blurring
radius = 1.4 # use "water radius"
print("N.B. Gaussian bluring assumes 1 A grid resolution!")
x = np.linspace(-3, 3, 7) # stepx = res = 1 A
gaus = gaussian(x, radius)
gaus3d = np.multiply.outer(gaus, np.multiply.outer(gaus, gaus))
# apply gaussian "blurring" to provide smoother surface
Href = signal.fftconvolve(Href, gaus3d, mode='same')
# normalize to std.
Href /= Href[mask_nonzero].std() # remove zeroes from std calc.
    
writeDX("test3_ref-all.dx", Href, edges)
writeDX("test3_ref-all_diff.dx", H-Href, edges)

trajectory length =  11852
number of dimension =  417
This took: 42.168 s.
trajectory length =  11852
number of dimension =  417
This took: 60.435 s.
trajectory length =  11852
number of dimension =  417
This took: 58.979 s.
trajectory length =  11852
number of dimension =  417
This took: 59.606 s.
trajectory length =  11852
number of dimension =  417
This took: 61.515 s.
trajectory length =  11852
number of dimension =  417
This took: 60.814 s.
trajectory length =  11852
number of dimension =  417
This took: 59.043 s.
trajectory length =  11852
number of dimension =  417
This took: 59.524 s.
trajectory length =  11852
number of dimension =  417
This took: 59.149 s.
trajectory length =  11852
number of dimension =  417
This took: 59.277 s.
trajectory length =  11852
number of dimension =  417
This took: 60.351 s.
trajectory length =  11852
number of dimension =  417
This took: 58.833 s.
trajectory length =  11852
number of dimension =  417
This took: 59.273 s.
trajectory length =  1185

In [530]:
H = np.empty((500,500,500), dtype=float)

In [497]:
# Is the normalization by std consistent enough to allow direct comparison between different simulations?
# volume/concentration/frames

# writeDX("test3_ref0_diff.dx", H - Href, edges)

In [508]:
view = nv.show_structure_file("TEM1/ref_1JWP+WAT+BEN.gro")
view.add_component("test3_ref-all_diff.dx")
view.component_1.clear_representations()
# view.component_1.add_surface(opacity=0.9, color="green", isolevelType="sigma", isovalue=0.5)
view.component_1.add_surface(opacity=0.5, color="red", isolevelType="value", isolevel=+10)
view.component_1.add_surface(opacity=0.5, color="blue", isolevelType="value", isolevel=-10)
# view.add_component("test3_ref-all_diff.dx")
# view.component_1.clear_representations()
# # view.component_1.add_surface(opacity=0.9, color="green", isolevelType="sigma", isovalue=0.5)
# view.component_1.add_surface(opacity=0.5, color="red", isolevelType="value", isolevel=+5)

In [509]:
view

The installed widget Javascript is the wrong version.
